In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%load_ext cudf.pandas
import cuml
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv', index_col='id')

In [3]:
train.shape

(750000, 11)

In [4]:
test.shape

(250000, 10)

## Data Understanding

In [5]:
train.sample(10)

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
id,,,,,,,,,,,
92728,Comedy Corner,Episode 18,64.57,Comedy,84.10,Wednesday,Afternoon,90.38,2.0,Neutral,46.93636
189441,Criminal Minds,Episode 66,<NA>,True Crime,93.94,Monday,Afternoon,<NA>,3.0,Negative,52.94834
559777,Mind & Body,Episode 92,12.39,Health,60.50,Sunday,Evening,84.2,3.0,Positive,8.05935
555138,Learning Lab,Episode 86,92.29,Education,52.26,Sunday,Afternoon,32.65,1.0,Positive,67.46362
103829,Sound Waves,Episode 75,62.51,Music,90.58,Monday,Evening,8.31,0.0,Negative,34.99625
708865,World Watch,Episode 86,66.96,News,58.17,Saturday,Afternoon,82.09,1.0,Positive,37.29073
179711,Funny Folks,Episode 60,60.19,Comedy,22.63,Thursday,Evening,35.62,3.0,Negative,58.79097
592900,Crime Chronicles,Episode 71,79.04,True Crime,84.36,Friday,Night,82.12,1.0,Positive,72.58319
684676,Life Lessons,Episode 20,23.29,Lifestyle,21.18,Monday,Afternoon,30.53,3.0,Positive,18.53835


In [6]:
train.describe()

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,662907.000000,750000.000000,603970.000000,749999.000000,750000.000000
mean,64.504738,59.859901,52.236449,1.348855,45.437406
std,32.969603,22.873098,28.451241,1.151130,27.138306
min,0.000000,1.300000,0.000000,0.000000,0.000000
25%,35.730000,39.410000,28.380000,0.000000,23.178350
50%,63.840000,60.050000,53.580000,1.000000,43.379460
75%,94.070000,79.530000,76.600000,2.000000,64.811580
max,325.240000,119.460000,119.910000,103.910000,119.970000


In [7]:
train.shape

(750000, 11)

In [8]:
test.shape

(250000, 10)

In [9]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   Podcast_Name                 750000 non-null  object
 1   Episode_Title                750000 non-null  object
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object
 6   Publication_Time             750000 non-null  object
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 88.8+ MB


In [10]:
train.isnull().sum()

Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [11]:
test.isnull().sum()

Podcast_Name                       0
Episode_Title                      0
Episode_Length_minutes         28736
Genre                              0
Host_Popularity_percentage         0
Publication_Day                    0
Publication_Time                   0
Guest_Popularity_percentage    48832
Number_of_Ads                      0
Episode_Sentiment                  0
dtype: int64

## Data Preparation

Reducing the memory space of the dataset by converting the num_cols into float32 and cat_cols into category

In [12]:
num_cols = test.select_dtypes(include=['float64']).columns
for col in num_cols:
    train[col] = train[col].astype('float32')
    test[col] = test[col].astype('float32')

In [13]:
cat_cols = train.select_dtypes(include=['object']).columns
for col in cat_cols:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [14]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   Podcast_Name                 750000 non-null  category
 1   Episode_Title                750000 non-null  category
 2   Episode_Length_minutes       662907 non-null  float32
 3   Genre                        750000 non-null  category
 4   Host_Popularity_percentage   750000 non-null  float32
 5   Publication_Day              750000 non-null  category
 6   Publication_Time             750000 non-null  category
 7   Guest_Popularity_percentage  603970 non-null  float32
 8   Number_of_Ads                749999 non-null  float32
 9   Episode_Sentiment            750000 non-null  category
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: category(6), float32(4), float64(1)
memory usage: 27.5 MB


In [15]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='median')
num_cols = test.select_dtypes(include=['float32']).columns
train[num_cols] = impute.fit_transform(train[num_cols])
test[num_cols] = impute.fit_transform(test[num_cols])

In [16]:
sentiments = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
train['Episode_Sentiment'] = train['Episode_Sentiment'].map(sentiments).astype('float32')
test['Episode_Sentiment'] = test['Episode_Sentiment'].map(sentiments).astype('float32')

## Feature engineering

## Count Encoding for high cardinality columns

In [17]:
from category_encoders import CountEncoder

ce = CountEncoder()
count_category = ['Podcast_Name', 'Genre', 'Episode_Title']
train[count_category] = ce.fit_transform(train[count_category])
test[count_category] = ce.transform(test[count_category])

In [18]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['Publication_Day', 'Publication_Time']
for colname in cat_cols:
    le = LabelEncoder()
    train[colname] = le.fit_transform(train[colname])
    test[colname] = le.transform(test[colname])

In [19]:
from sklearn.preprocessing import KBinsDiscretizer

kmeans = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
train['Kmeans_bins'] = kmeans.fit_transform(train[['Episode_Length_minutes']])
test['Kmeans_bins'] = kmeans.transform(test[['Episode_Length_minutes']])

In [20]:
for col in ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Episode_Length_minutes']:
    for k in range(0, 3):
        new_round_col = f"{col}_round{k}"
        train[new_round_col] = train[col].round(k)
        test[new_round_col] = test[col].round(k)

In [21]:
train['Guest_Popularity_percentage'] =  train['Guest_Popularity_percentage'].clip(upper=100)
train['Host_Popularity_percentage'] =  train['Host_Popularity_percentage'].clip(upper=100)
test['Guest_Popularity_percentage'] =  test['Guest_Popularity_percentage'].clip(upper=100)
test['Host_Popularity_percentage'] =  test['Host_Popularity_percentage'].clip(upper=100)


In [22]:
train.columns

Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'Kmeans_bins',
       'Host_Popularity_percentage_round0',
       'Host_Popularity_percentage_round1',
       'Host_Popularity_percentage_round2',
       'Guest_Popularity_percentage_round0',
       'Guest_Popularity_percentage_round1',
       'Guest_Popularity_percentage_round2', 'Episode_Length_minutes_round0',
       'Episode_Length_minutes_round1', 'Episode_Length_minutes_round2'],
      dtype='object')

In [23]:
features = ['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre', 'Publication_Day', 'Publication_Time',
       'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment', 'Kmeans_bins', 'Host_Popularity_percentage_round0', 'Host_Popularity_percentage_round1',
       'Host_Popularity_percentage_round2', 'Guest_Popularity_percentage_round0', 'Guest_Popularity_percentage_round1',
       'Guest_Popularity_percentage_round2', 'Episode_Length_minutes_round0', 'Episode_Length_minutes_round1', 'Episode_Length_minutes_round2']

In [24]:
interact = []
for i, c1 in enumerate(features):
    for c2 in (features[i+1:]):
        n = f'{c1}_{c2}'
        train[n] = train[c1] * train[c2]
        test[n] = test[c1] * test[c2]
        interact.append(n)

print(f"There are {len(interact)} interaction features:")
print( interact )

There are 190 interaction features:
['Podcast_Name_Episode_Title', 'Podcast_Name_Episode_Length_minutes', 'Podcast_Name_Genre', 'Podcast_Name_Publication_Day', 'Podcast_Name_Publication_Time', 'Podcast_Name_Host_Popularity_percentage', 'Podcast_Name_Guest_Popularity_percentage', 'Podcast_Name_Number_of_Ads', 'Podcast_Name_Episode_Sentiment', 'Podcast_Name_Kmeans_bins', 'Podcast_Name_Host_Popularity_percentage_round0', 'Podcast_Name_Host_Popularity_percentage_round1', 'Podcast_Name_Host_Popularity_percentage_round2', 'Podcast_Name_Guest_Popularity_percentage_round0', 'Podcast_Name_Guest_Popularity_percentage_round1', 'Podcast_Name_Guest_Popularity_percentage_round2', 'Podcast_Name_Episode_Length_minutes_round0', 'Podcast_Name_Episode_Length_minutes_round1', 'Podcast_Name_Episode_Length_minutes_round2', 'Episode_Title_Episode_Length_minutes', 'Episode_Title_Genre', 'Episode_Title_Publication_Day', 'Episode_Title_Publication_Time', 'Episode_Title_Host_Popularity_percentage', 'Episode_Titl

In [25]:
from sklearn.model_selection import KFold
from cuml.metrics import mean_squared_error
from xgboost import XGBRegressor

n_folds = 10
test_preds = np.zeros(len(test))

kf = KFold(n_splits=n_folds, shuffle=True, random_state=34)

for fold, (train_idx, valid_idx) in enumerate(kf.split(train)):
    X_train, X_valid = train.iloc[train_idx][features], train.iloc[valid_idx][features]
    y_train, y_valid = train.iloc[train_idx]['Listening_Time_minutes'], train.iloc[valid_idx]['Listening_Time_minutes']

    xgb = XGBRegressor(
        n_estimators=600, 
        learning_rate=0.07, 
        reg_lambda=8.20664, 
        max_depth=11,
        colsample_bytree=0.5,
        subsample=0.8,
        device='cuda',
        tree_method='gpu_hist',
        eval_metric='rmse',
        verbosity=0,
        random_state=34
    )
    xgb.fit(X_train, y_train)
    
    valid_pred = xgb.predict(X_valid)
    fold_rmse = np.sqrt(mean_squared_error(y_valid, valid_pred))
    print(f"Fold {fold + 1} RMSE: {fold_rmse:.2f}")
    
    test_preds += xgb.predict(test[features]) / n_folds


Fold 1 RMSE: 12.82
Fold 2 RMSE: 12.81
Fold 3 RMSE: 12.75
Fold 4 RMSE: 12.69
Fold 5 RMSE: 12.66
Fold 6 RMSE: 12.65
Fold 7 RMSE: 12.70
Fold 8 RMSE: 12.74
Fold 9 RMSE: 12.78
Fold 10 RMSE: 12.74


## Final Submission

In [26]:
sub = pd.read_csv('/kaggle/input/playground-series-s5e4/sample_submission.csv')
sub['Listening_Time_minutes'] = test_preds
sub.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
